In [24]:
import json
import pandas as pd

In [25]:
with open('data/female-types-with-parameters.json') as f:
    types = json.load(f)
with open('data/female-types.json') as f:
    female_structure = json.load(f)
with open('data/type-parameters.json') as f:
    female_params = json.load(f)

In [26]:
from ast import literal_eval

def get_value(list_values,column):
    interim=[]
    for i in range(len(list_values)):
        interim.append(list_values[i][column])
    return interim

def transform_jsons_flatten_df(gender_types_with_parameters,
                               gender_types,
                               type_parameters):
    gender_types_with_parameters_df=pd.DataFrame(gender_types_with_parameters)
    gender_types_df=pd.DataFrame(gender_types)
    type_parameters_df=pd.DataFrame(type_parameters)
    # merge to get parameter for each type
    df=gender_types_df.merge(gender_types_with_parameters_df, left_on='type', right_on='clothes_type')
    #flatten df to get more vivid view
    df['clothesParameters']=df.parameters.apply(lambda x: get_value(x,'name'))
    df=df[['category','subcategory','type','clothesParameters']].explode('clothesParameters')
    # merge to get value of parameter
    df=df.merge(type_parameters_df,left_on=['category','clothesParameters'],
           right_on=['type','name'])
    df=df[['category','subcategory','type_x','clothesParameters','values']].rename(columns={'type_x':'type'}).explode('values')
    return df

In [27]:
df_flatten=transform_jsons_flatten_df(gender_types_with_parameters=types,
                           gender_types=female_structure,
                           type_parameters=female_params)

## assign priority and intersect with rules

In [29]:
with open('data/male-body-type-rules.json') as f:
    body_rules = json.load(f)

In [30]:
body_rules_df=pd.DataFrame(body_rules)

In [31]:
def text_to_priority_number(text):
    if text=='high':
        return float(2)
    if text=='low':
        return float(0.1)
    if text=='excluded':
        return float(0.0000001)
    else:
        return float(1)

In [32]:
body_rules_df['priorityProb']=body_rules_df.priorityType.apply(text_to_priority_number)
body_rules_df = body_rules_df[body_rules_df['key']=='rectangle']
df_flatten['priorityProb'] = 1.0

In [35]:
# merge where paramName=None
df_flatten=df_flatten.merge(body_rules_df[body_rules_df['ParamName'].isnull()],left_on=['type'],right_on=['clothesType'],how='left')\
[['category', 'subcategory', 'type', 'clothesParameters', 'values', 'priorityProb_x', 'priorityProb_y']]
df_flatten['priorityProb_y']=df_flatten['priorityProb_y'].fillna(1)
df_flatten['priorityProb']=df_flatten['priorityProb_y']*df_flatten['priorityProb_x']
df_flatten.drop(columns=['priorityProb_x','priorityProb_y'],inplace=True)
# merge where not None
df_flatten=df_flatten.merge(body_rules_df[~body_rules_df['ParamName'].isnull()],
                            left_on=['type','clothesParameters','values'],
                            right_on=['clothesType','ParamName','ParamValues'],how='left')\
[['category', 'subcategory', 'type', 'clothesParameters', 'values', 'priorityProb_x', 'priorityProb_y']]
df_flatten['priorityProb_y']=df_flatten['priorityProb_y'].fillna(1)
df_flatten['priorityProb']=df_flatten['priorityProb_y']*df_flatten['priorityProb_x']
df_flatten.drop(columns=['priorityProb_x','priorityProb_y'],inplace=True)


In [36]:
df_flatten

,category,subcategory,type,clothesParameters,values,priorityProb
0,top,hoodies&sweatshirts,sweatshirt,silhouette,slim,1.0
1,top,hoodies&sweatshirts,sweatshirt,silhouette,regular,1.0
2,top,hoodies&sweatshirts,sweatshirt,silhouette,oversize,1.0
3,top,hoodies&sweatshirts,hoodie,silhouette,slim,2.0
4,top,hoodies&sweatshirts,hoodie,silhouette,regular,2.0
...,...,...,...,...,...,...
160,footwear,boots,ankle_boots,heel_type,flat,1.0
161,footwear,pumps,dress_sandals,heel_type,stiletto,1.0
162,footwear,pumps,dress_sandals,heel_type,platform,1.0
163,footwear,pumps,dress_sandals,heel_type,heel,1.0


In [19]:
body_rules_df

,key,clothesType,priorityType,ParamName,ParamValues,priorityProb
0,rectangle,hoodie,high,None,None,2.0
1,rectangle,sweater,high,neckline,V-neck,2.0
2,rectangle,sweater,low,silhouette,slim,0.1
3,rectangle,cardigan,high,zipper_type,buttons,2.0
4,rectangle,cardigan,low,silhouette,oversize,0.1
...,...,...,...,...,...,...
124,trapezoid,long_sleeve_T-shirt,high,silhouette,regular,2.0
125,trapezoid,denim_shirt,high,silhouette,slim,2.0
126,trapezoid,long_sleeve_shirt,high,silhouette,slim,2.0
127,trapezoid,short_sleeve_shirt,high,silhouette,slim,2.0


In [ ]:
body_rules_df

In [155]:
df_flatten=df_flatten.merge(body_rules_df[~body_rules_df['ParamName'].isnull()],
                            left_on=['type','clothesParameters','values'],
                            right_on=['clothesType','ParamName','ParamValues'],how='left')\
[['category', 'subcategory', 'type', 'clothesParameters', 'values', 'priorityProb_x', 'priorityProb_y']]



In [171]:
df_flatten[df_flatten['priorityProb_y'].isna()].priorityProb_y

0     NaN
3     NaN
14    NaN
15    NaN
30    NaN
       ..
223   NaN
224   NaN
225   NaN
226   NaN
227   NaN
Name: priorityProb_y, Length: 123, dtype: float64

In [ ]:
df_flatten['priorityProb_y']=df_flatten['priorityProb_y'].fillna(1)
df_flatten['priorityProb']=df_flatten['priorityProb_y']+df_flatten['priorityProb_x']
df_flatten['priorityProb']=df_flatten['priorityProb']/df_flatten['priorityProb'].max()

In [ ]:
df_flatten.drop(columns=['priorityProb_x','priorityProb_y'],inplace=True)

In [151]:
df_flatten

,category,subcategory,type,clothesParameters,values,priorityProb_x,priorityProb_y,priorityProb
0,top,hoodies&sweatshirts,sweatshirt,silhouette,slim,0.5,1.0,1.0
1,top,hoodies&sweatshirts,sweatshirt,silhouette,regular,0.5,1.0,1.0
2,top,hoodies&sweatshirts,sweatshirt,silhouette,oversize,0.5,0.1,0.4
3,top,hoodies&sweatshirts,hoodie,silhouette,slim,0.5,1.0,1.0
4,top,hoodies&sweatshirts,hoodie,silhouette,regular,0.5,1.0,1.0
...,...,...,...,...,...,...,...,...
223,footwear,boots,ankle_boots,heel_type,flat,0.5,1.0,1.0
224,footwear,pumps,dress_sandals,heel_type,stiletto,0.5,1.0,1.0
225,footwear,pumps,dress_sandals,heel_type,platform,0.5,1.0,1.0
226,footwear,pumps,dress_sandals,heel_type,heel,0.5,1.0,1.0


# Черновик

In [45]:
def rule(text):
    if text == None:
        return text
    return text.split(':')[1]

In [25]:
body_rules_df['paramName']=body_rules_df.paramType.apply(rule)
body_rules_df['paramValues'] =body_rules_df.paramType.apply(rule)
body_rules_df=body_rules_df.drop(columns=['paramType'])
body_rules_df.to_json('male-body-type-rules.json',indent=4)